# sea surface temperature fields

In [24]:
import s3fs; import xarray as xr; import numpy as np
import pandas as pd; 
import dask.array as da
import ocetrac

import matplotlib.pyplot as plt; import cartopy.crs as ccrs

import warnings; import expectexception
warnings.filterwarnings('ignore')

import netCDF4 as nc; import datetime as dt
import scipy

import intake; import pprint
# Allow multiple lines per cell to be displayed without print (default is just last line)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Enable more explicit control of DataFrame display (e.g., to omit annoying line numbers)
from IPython.display import HTML

import dask
from distributed import Client
from ncar_jobqueue import NCARCluster

In [25]:
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)

In [26]:
subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')
member_id_list = subset.df.member_id.unique()
print(member_id_list)

['r1i1001p1f1' 'r2i1021p1f1' 'r3i1041p1f1' 'r4i1061p1f1' 'r5i1081p1f1'
 'r6i1101p1f1' 'r7i1121p1f1' 'r8i1141p1f1' 'r9i1161p1f1' 'r10i1181p1f1'
 'r1i1231p1f1' 'r2i1231p1f1' 'r3i1231p1f1' 'r4i1231p1f1' 'r5i1231p1f1'
 'r6i1231p1f1' 'r7i1231p1f1' 'r8i1231p1f1' 'r9i1231p1f1' 'r10i1231p1f1'
 'r1i1251p1f1' 'r2i1251p1f1' 'r3i1251p1f1' 'r4i1251p1f1' 'r5i1251p1f1'
 'r6i1251p1f1' 'r7i1251p1f1' 'r8i1251p1f1' 'r9i1251p1f1' 'r10i1251p1f1'
 'r1i1281p1f1' 'r2i1281p1f1' 'r3i1281p1f1' 'r4i1281p1f1' 'r5i1281p1f1'
 'r6i1281p1f1' 'r7i1281p1f1' 'r8i1281p1f1' 'r9i1281p1f1' 'r10i1281p1f1'
 'r1i1301p1f1' 'r2i1301p1f1' 'r3i1301p1f1' 'r4i1301p1f1' 'r5i1301p1f1'
 'r6i1301p1f1' 'r7i1301p1f1' 'r8i1301p1f1' 'r9i1301p1f1' 'r10i1301p1f1'
 'r1i1011p1f2' 'r2i1031p1f2' 'r3i1051p1f2' 'r4i1071p1f2' 'r5i1091p1f2'
 'r6i1111p1f2' 'r7i1131p1f2' 'r8i1151p1f2' 'r9i1171p1f2' 'r10i1191p1f2'
 'r11i1231p1f2' 'r12i1231p1f2' 'r13i1231p1f2' 'r14i1231p1f2'
 'r15i1231p1f2' 'r16i1231p1f2' 'r17i1231p1f2' 'r18i1231p1f2'
 'r19i1231p1f2' 'r20

In [5]:
# Create our NCAR Cluster - which uses PBSCluster under the hood
cluster = NCARCluster()

# Spin up 20 workers
cluster.scale(20)

# Assign the cluster to our Client
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cassiacai/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cassiacai/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.8:39045,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cassiacai/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
cat = intake.open_esm_datastore('/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json')

In [8]:
cat

,unique
component,6
stream,26
case,200
member_id,100
variable,1906
start_time,157
end_time,180
time_range,163
long_name,1800
units,184


In [9]:
cat.search(component='atm', long_name=['temperature*', 'Temperature*']).df.long_name.unique()

array(['Temperature', 'Temperature at   10 mbar pressure surface',
       'Temperature at   50 mbar pressure surface',
       'Temperature at  100 mbar pressure surface',
       'Temperature at 1000 mbar pressure surface',
       'Temperature at 200 mbar pressure surface',
       'Temperature at  200 mbar pressure surface',
       'Temperature at 500 mbar pressure surface',
       'Temperature at  500 mbar pressure surface',
       'Temperature at 700 mbar pressure surface',
       'Temperature at  700 mbar pressure surface',
       'Temperature at 850 mbar pressure surface',
       'Temperature at  850 mbar pressure surface',
       'Lowest model level temperature', 'Reference height temperature',
       'Minimum reference height temperature over output period',
       'Maximum reference height temperature over output period',
       'Surface temperature (radiative)',
       'Minimum surface temperature over output period',
       'Maximum surface temperature over output period',
    

In [10]:
memberid = str(r2i1021p1f1)
subset = cat.search(component='atm',frequency='month_1',long_name='sea surface temperature',experiment='historical',member_id= memberid)

In [11]:
subset

,unique
component,1
stream,1
case,1
member_id,1
variable,1
start_time,17
end_time,17
time_range,17
long_name,1
units,1


In [12]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dsets = subset.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


In [13]:
dsets.keys()

dict_keys(['atm.historical.cam.h0.cmip6.SST'])

In [14]:
ds = dsets['atm.historical.cam.h0.cmip6.SST']

In [15]:
SST = ds.SST.isel(member_id=0)
SST.load()

<xarray.DataArray 'SST' (time: 1980, lat: 192, lon: 288)>
array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.33493, 271.33496, 271.33496, ..., 271.33487, 271.33487,
         271.33493],
        [271.33932, 271.33932, 271.33936, ..., 271.33932, 271.33932,
         271.33932],
        [271.34225, 271.34225, 271.34225, ..., 271.3423 , 271.34225,
         271.34225]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
...
        [271.3928 , 271.39285, 271.3929 , ..., 271.3927 , 271.39273,
         271.39276],
        [271.40018, 271.4002 , 271.40024, ..., 271.40018, 271.40018,
         271.40018],
        [271.4055 , 271.4055 , 271.4055 , ..., 271.4055 , 271.4055 ,
         271.4055 ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.3838 , 271.38385, 271.3839 , ..., 271.3837 , 271.3837 ,
         271.38376],
        [271.3916 , 271.39166, 271.3917 , ..., 271.39154, 271.39154,
         271.39157],
        [271.39667, 271.39664, 271.39664, ..., 271.39667, 271.39667,
         271.39667]]], dtype=float32)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r2i1021p1f1'
Attributes:
    units:         K
    long_name:     sea surface temperature
    cell_methods:  time: mean

### detrending

In [17]:
start_val = 0
end_val = 1980

In [18]:
dyr = SST.time.dt.year + (SST.time.dt.month-0.5)/12
dyr = dyr[start_val:end_val]
model = np.array([np.ones(len(dyr))] + [dyr-np.mean(dyr)] + [np.sin(2*np.pi*dyr)] + [np.cos(2*np.pi*dyr)] + [np.sin(4*np.pi*dyr)] + [np.cos(4*np.pi*dyr)])
pmodel = np.linalg.pinv(model)
model_da = xr.DataArray(model.T, dims=['time','coeff'], coords={'time':SST.time.values[start_val:end_val], 'coeff':np.arange(1,7,1)}) 
pmodel_da = xr.DataArray(pmodel.T, dims=['coeff','time'], coords={'coeff':np.arange(1,7,1), 'time':SST.time.values[start_val:end_val]})
sst_mod = xr.DataArray(pmodel_da.dot(SST), dims=['coeff','lat','lon'], coords={'coeff':np.arange(1,7,1), 'lat':SST.lat.values, 'lon':SST.lon.values})
mean = model_da[:,0].dot(sst_mod[0,:,:])
trend = model_da[:,1].dot(sst_mod[1,:,:])
seas = model_da[:,2:].dot(sst_mod[2:,:,:])
ssta_notrend = SST-model_da.dot(sst_mod)
detrended = ssta_notrend

In [19]:
detrended

<xarray.DataArray (time: 1980, lat: 192, lon: 288)>
array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.00778218, -0.00778454, -0.00784484, ..., -0.00775895,
         -0.00778664, -0.00776694],
        [-0.00950761, -0.00954477, -0.00955207, ..., -0.00943695,
         -0.00944919, -0.00947577],
        [-0.00978937, -0.00978853, -0.00978832, ..., -0.00976581,
         -0.00979354, -0.00979142]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
...
        [ 0.019167  ,  0.01920743,  0.01923676, ...,  0.01912945,
          0.01913831,  0.01914027],
        [ 0.02244937,  0.02245658,  0.02246961, ...,  0.02248444,
          0.02248199,  0.02246713],
        [ 0.02476496,  0.02476495,  0.0247655 , ...,  0.02476291,
          0.0247636 ,  0.02476474]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.01120587,  0.01127215,  0.01132228, ...,  0.01110126,
          0.01110259,  0.01115526],
        [ 0.01448125,  0.01452595,  0.01454516, ...,  0.01443189,
          0.01443814,  0.01446169],
        [ 0.0165663 ,  0.01653674,  0.01653784, ...,  0.01656087,
          0.01656303,  0.01656516]]])
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-02-01 00:00:00 ... 2015-01-01 00:00:00
    member_id  <U11 'r2i1021p1f1'

In [21]:
detrended.to_netcdf('ens0_detrended.nc')